In [6]:
class MyAverage():
    def __init__(self):
        self.numbers = []
    
    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count

my_class = MyAverage()
my_class.add(10)

15.0

## Вместо класса используем замыкание

In [18]:
def closure():
    numbers = []
    def inner(n):
        numbers.append(n)
        total = sum(numbers)
        count = len(numbers)
        return total / count
    return inner

a = closure()
print(a.__closure__)
print(a.__code__.co_freevars)
a(10)

(<cell at 0x00000264DF69EE58: list object at 0x00000264DF65A688>,)
('numbers',)


10.0

## Избавимся от списка

In [28]:
def averager():
    total = 0
    count = 0
    def inner(number):
        nonlocal total
        nonlocal count
        total += number
        count += 1
        return total / count
    return inner

a2 = averager()
print(a2.__code__.co_freevars)
print(a2.__closure__)
a2(10)

('count', 'total')
(<cell at 0x00000264DF66AE28: int object at 0x00007FFA3153A170>, <cell at 0x00000264DF66A078: int object at 0x00007FFA3153A170>)


10.0

In [30]:
class MyAverager():
    def __init__(self):
        self.total = 0
        self.count = 0
    
    def func(self, number):
        self.total += number
        self.count += 1
        return self.total / self.count

a = MyAverager()
print(a.func(10))

10.0


## Добавим счётчик

In [15]:
from time import perf_counter

class Timer():
    def __init__(self):
        self.start = perf_counter()
    
    def counter(self):
        return perf_counter() - self.start

t1 = Timer()
t1.counter()

3.749999996216502e-05

In [16]:
t1.counter()

3.429275299999972

## Упростим класс с помощью метода __call__

In [21]:
from time import perf_counter
class Timer():
    def __init__(self):
        self.start = perf_counter()
    
    def __call__(self):
        return perf_counter() - self.start

t2 = Timer()
print(t2())

5.9699999951590144e-05


## Сделаем тоже самое с помощью замыкания

In [24]:
def closure():
    start = perf_counter()
    def inner():
        return perf_counter() - start
    return inner

a = closure()
print(a())

4.5700000100623583e-05


## Сделаем простой счётчик

In [28]:
def counter(initial_value = 0):
    def inc(increment = 1):
        nonlocal initial_value
        initial_value += increment
        return initial_value
    return inc

a1 = counter()
print(a1())
print(a1())

1
2


## Будем отсчитывать, сколько раз отработала функция

In [29]:
def counter(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print("{0} has been called {1} times".format(fn.__name__, count))
        return fn(*args, **kwargs)
    return inner

def add(a, b):
    return a + b

def mult(a, b):
    return a * b

In [33]:
counter_add = counter(add)
print(counter_add(10, 34))

add has been called 1 times
44


## Расширим, будем сохранять в словарь название функции и сколько раз она вызывалась

In [34]:
dicter = {}


def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        dicter[fn.__name__] = count
        return fn(*args, **kwargs)
    return inner


def add(x, y):
    return x + y


def mult(x, y):
    return x * y


a = counter(add)
b = counter(mult)
print(a(3, 4))
print(a(3, 4))
print(a(3, 4))
print(b(6, 6))
print(b(6, 6))
print(dicter)

7
7
7
36
36
{'add': 3, 'mult': 2}


## Можем добавить словарь как параментр функции

In [36]:
def counter(fn, counters):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        counters[fn.__name__] = count
        return fn(*args, **kwargs)
    return inner

c = {}

a = counter(add, c)
b = counter(mult, c)
print(a(3, 4))
print(a(3, 4))
print(a(3, 4))
print(b(6, 6))
print(b(6, 6))
print(dicter)

7
7
7
36
36
{'add': 3, 'mult': 2}


In [41]:
def fact(n):
    start = 1
    for x in range(2, n + 1):
        start *= x
    return start


counter_add = counter(fact, c)
print(counter_add(5))
print(counter_add(7))
print(c)

120
5040
{'add': 3, 'mult': 2, 'fact': 2}


## Пришли к декораторам

In [45]:
fact = counter(fact, c)
add = counter(add, c)
print(fact(5))
print(add(5, 56))
print(c)

120
61
{'add': 2, 'mult': 2, 'fact': 3, 'inner': 1}
